<a href="https://colab.research.google.com/github/crea0414/1st_CVDL/blob/master/Day019_Inception_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 『本次練習內容』
#### 學習如何搭建Inception Block

## 『本次練習目的』
  #### 了解Inceotion原理
  #### 了解如何導入Inception block到原本架構中

In [1]:
import tensorflow as tf
tf.__version__

'2.3.0'

## ConvBN

In [2]:
# Conv2d witout "bias"
# BatchNormalization shoud set scale False (not applied gamma) when next activation is tf.nn.relu

class Conv2D_BN(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding='same', activation='relu', name=None, **kwargs):
        super(Conv2D_BN, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.activation = activation
    
    def build(self, input_shape):
        self.conv2d = tf.keras.layers.Conv2D(self.filters, self.kernel_size,
                                             self.strides, self.padding, use_bias=False)
        self.batchnorm = tf.keras.layers.BatchNormalization(axis=-1, scale=False)
        self.activate_layer = tf.keras.layers.Activation(self.activation)

    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.batchnorm(x)
        x = self.activate_layer(x)
        return x


In [3]:
#check
tf.keras.backend.clear_session()
conv2d_bn = Conv2D_BN(64, (3,3))
model = tf.keras.Sequential([conv2d_bn])
output = model(tf.random.normal((1, 224, 224, 1)))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_bn (Conv2D_BN)        (1, 224, 224, 64)         768       
Total params: 768
Trainable params: 640
Non-trainable params: 128
_________________________________________________________________


In [4]:
#with bias and bn applied scale 
print((3*3*1+1)*64+64*4)
#without bias and bn scale off
(3*3*1)*64+64*3

896


768

In [5]:
#with bias
print(f'parameters_count: conv-{(3*3*1+1)*64} batchnorm-(output_chanels*(gamma+betta+moving_mean+moving_var){64*4}')
#without bias
print(f'parameters_count: conv-{(3*3*1)*64} batchnorm-(output_chanels*(betta+moving_mean+moving_var){64*3}')

parameters_count: conv-640 batchnorm-(output_chanels*(gamma+betta+moving_mean+moving_var)256
parameters_count: conv-576 batchnorm-(output_chanels*(betta+moving_mean+moving_var)192


#Inception V2 Block

In [6]:
class InceptionV2Block(tf.keras.layers.Layer):
    def __init__(self, filters_spec = ((64,), (96, 128), (16, 32), (32,)), **kwargs):
        super(InceptionV2Block, self).__init__(**kwargs)
        self.branch_0 = filters_spec[0][0]
        self.branch_1 = filters_spec[1]
        self.branch_2 = filters_spec[2]
        self.branch_3 = filters_spec[3][0]

    def build(self, input_shape):
        self.conv2d_b0 = Conv2D_BN(self.branch_0, kernel_size=(1, 1))
        self.conv2d_b1 = Conv2D_BN(self.branch_1[0], (1, 1))
        self.conv2d_b1_1 = Conv2D_BN(self.branch_1[1], (3, 3))
        self.conv2d_b2 = Conv2D_BN(self.branch_2[0], (1, 1))
        self.conv2d_b2_1 = Conv2D_BN(self.branch_2[1], (3, 3))
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')
        self.conv2d_b3 = Conv2D_BN(self.branch_3, (1, 1))
    
    def call(self, inputs):
        b0 = self.conv2d_b0(inputs)
        b1 = self.conv2d_b1(inputs)
        b1_1 = self.conv2d_b1_1(b1)
        b2 = self.conv2d_b2(inputs)
        b2_1 = self.conv2d_b2_1(b2)
        b3 = self.max_pool(inputs)
        b3_1 = self.conv2d_b3(b3)
        
        return tf.concat([b0, b1_1, b2_1, b3_1], axis=-1)

In [7]:
#check 
tf.keras.backend.clear_session()

inception_block = InceptionV2Block()
model = tf.keras.Sequential([inception_block])
output = model(tf.random.normal((1, 224, 224, 1)))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v2block (Inception (1, 224, 224, 256)        116512    
Total params: 116,512
Trainable params: 115,776
Non-trainable params: 736
_________________________________________________________________


In [8]:
#with bias and bn applied scale 
in_ch = 1
(1*1*in_ch)*64 + 64*3 + \
(1*1*in_ch)*96 + 96*3 + (3*3*96)*128 + 128*3 + \
(1*1*in_ch)*16 + 16*3 + (5*5*16)*32 + 32*3 + \
(1*1*in_ch)*32 + 32*3 , (64*2 + 96*2 + 128*2 + 16*2 + 32*2 + 32*2 )

(124704, 736)

In [9]:
#without bias and bn scale off
in_ch = 1
(1*1*in_ch+1)*64 + 64*4 + \
(1*1*in_ch+1)*96 + 96*4 + (3*3*96+1)*128 + 128*4 + \
(1*1*in_ch+1)*16 + 16*4 + (5*5*16+1)*32 + 32*4 + \
(1*1*in_ch+1)*32 + 32*4 , (64*2 + 96*2 + 128*2 + 16*2 + 32*2 + 32*2 )

(125440, 736)

# Inception V3 Block

In [10]:
class InceptionV3Block(tf.keras.layers.Layer):
    def __init__(self, filters_spec = ((64,), (96, 128), (16, 32), (32,)), **kwargs):
        super(InceptionV3Block, self).__init__(**kwargs)
        self.branch_0 = filters_spec[0][0]
        self.branch_1 = filters_spec[1]
        self.branch_2 = filters_spec[2]
        self.branch_3 = filters_spec[3][0]
    
    def build(self, input_shape):
        self.conv2d_0 = Conv2D_BN(self.branch_0, (1, 1))
        self.conv2d_1 = Conv2D_BN(self.branch_1[0], (1, 1))
        self.conv2d_1_1 = Conv2D_BN(self.branch_1[1], (1, 3))
        self.conv2d_1_2 = Conv2D_BN(self.branch_1[1], (3, 1))
        self.conv2d_2 = Conv2D_BN(self.branch_2[0], (1, 1))
        self.conv2d_2_1 = Conv2D_BN(self.branch_2[1], (1, 3))
        self.conv2d_2_2 = Conv2D_BN(self.branch_2[1], (3, 1))
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')
        self.conv2d_3 = Conv2D_BN(self.branch_3, (1, 1))
        
    def call(self, inputs):
        b0 = self.conv2d_0(inputs)
        b1 = self.conv2d_1(inputs)
        b1_1 = self.conv2d_1_1(b1)
        b1_2 = self.conv2d_1_2(b1_1)
        b2 = self.conv2d_2(inputs)
        b2_1 = self.conv2d_2_1(b2)
        b2_2 = self.conv2d_2_2(b2_1)
        b3 = self.max_pool(inputs)
        b3_1 = self.conv2d_3(b3)
        return tf.concat([b0, b1_2, b2_2, b3_1], axis=-1)

In [11]:
#check 
tf.keras.backend.clear_session()

inception_block = InceptionV3Block()
model = tf.keras.Sequential([inception_block])
output = model(tf.random.normal((1, 224, 224, 1)))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3block (Inception (1, 224, 224, 256)        92416     
Total params: 92,416
Trainable params: 91,360
Non-trainable params: 1,056
_________________________________________________________________


In [12]:
#count total params
in_ch = 1
(1*1*in_ch)*64 + 3*64 + \
(1*1*in_ch)*96 + 3*96 + (1*3*96)*128 + 3*128 + (3*1*128)*128 + 3*128 + \
(1*1*in_ch)*16 + 3*16 + (1*3*16)*32 + 3*32 + (3*1*32)*32 + 3*32 + \
(1*1*in_ch)*32 + 3*32 

92416

# VGG16

In [13]:
vgg16 = tf.keras.applications.VGG16()
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
class VggBlockConvBN2(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=(3, 3), **kwargs):
        super(VggBlockConvBN2, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
    def build(self, input_shape):
        self.conv1 = Conv2D_BN(self.filters, self.kernel_size, padding='same')
        self.conv2 = Conv2D_BN(self.filters, self.kernel_size, padding='same')
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        return self.max_pool(x)

class VggBlockConvBN3(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=(3, 3), **kwargs):
        super(VggBlockConvBN3, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
    def build(self, input_shape):
        self.conv1 = Conv2D_BN(self.filters, self.kernel_size, padding='same')
        self.conv2 = Conv2D_BN(self.filters, self.kernel_size, padding='same')
        self.conv3 = Conv2D_BN(self.filters, self.kernel_size, padding='same')
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        return self.max_pool(x)

In [15]:
tf.keras.backend.clear_session()
vgg_block2 = VggBlockConvBN2(64)
model = tf.keras.Sequential([vgg_block2])
_ = model(tf.random.normal((1, 224, 224, 1)))

vgg_block3 = VggBlockConvBN3(64)
model2 = tf.keras.Sequential([vgg_block3])
_ = model2(tf.random.normal((1, 224, 224, 1)))

model.summary(), model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg_block_conv_b_n2 (VggBloc (1, 112, 112, 64)         37824     
Total params: 37,824
Trainable params: 37,568
Non-trainable params: 256
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg_block_conv_b_n3 (VggBloc (1, 112, 112, 64)         74880     
Total params: 74,880
Trainable params: 74,496
Non-trainable params: 384
_________________________________________________________________


(None, None)

In [16]:
params = (3*3*1)*64 + 64*3 + (3*3*64)*64 + 64*3
params2 = (3*3*1)*64 + 64*3 + (3*3*64)*64 + 64*3 + (3*3*64)*64 + 64*3 
params, params2

(37824, 74880)

In [17]:
class MyVGG16BN(tf.keras.Model):
    def __init__(self, include_top=True, pooling='max', classes=1000, **kwargs):
        super(MyVGG16BN, self).__init__(**kwargs)
        self.block1 = VggBlockConvBN2(64)
        self.block2 = VggBlockConvBN2(128)
        self.block3 = VggBlockConvBN3(256)
        self.block4 = VggBlockConvBN3(512)
        self.block5 = VggBlockConvBN3(512)
        self.classes = classes
        self.include_top = include_top
        self.pooling = pooling
        
        if self.include_top:
            self.flatten = tf.keras.layers.Flatten()
            self.dense1 = tf.keras.layers.Dense(4096)
            self.bn1 = tf.keras.layers.BatchNormalization()
            self.dense2 = tf.keras.layers.Dense(4096)
            self.bn2 = tf.keras.layers.BatchNormalization()
            self.dense3 = tf.keras.layers.Dense(self.classes)
        else:
            if self.pooling == 'max':
                self.pooling_layer = tf.keras.layers.GlobalMaxPooling2D()
            else:
                self.pooling_layer = tf.keras.layers.GlobalAveragePooling2D()
        
    def call(self, inputs):
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)

        if self.include_top:
            x = self.flatten(x)
            x = self.dense1(x)
            x = self.bn1(x)
            x = tf.nn.relu(x)
            x = self.dense2(x)
            x = self.bn2(x)
            x = tf.nn.relu(x)
            x = self.dense3(x)
            return tf.nn.softmax(x)
        else:
            x = self.pooling_layer(x)
            return x

In [18]:
tf.keras.backend.clear_session()
my_vgg16 = MyVGG16BN(include_top=True)
output = my_vgg16(tf.random.normal((1, 224, 224, 3)))
my_vgg16.summary()

Model: "my_vg_g16bn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg_block_conv_b_n2 (VggBloc multiple                  38976     
_________________________________________________________________
vgg_block_conv_b_n2_1 (VggBl multiple                  221952    
_________________________________________________________________
vgg_block_conv_b_n3 (VggBloc multiple                  1476864   
_________________________________________________________________
vgg_block_conv_b_n3_1 (VggBl multiple                  5902848   
_________________________________________________________________
vgg_block_conv_b_n3_2 (VggBl multiple                  7082496   
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  

In [19]:
#batch params count

batch_cnt = 64*4*2 + 128*4*2 + 256*4*3 + 512*4*3*2 + 16384 *2
batch_cnt, 138407208 - batch_cnt + (64*2 + 128*2 + 256*2 + 512*2)

(49664, 138359464)

# Custom Vgg16
- conv_layer in block3 >> inception_v2 block
- conv_layer in block5 >> inception_v3 block

In [20]:
filter_spec = ((64, ), (96, 128), (16, 32), (32,))
filter_spec1 = ((128, ), (192, 256), (32, 64), (64,))

In [21]:
filter_spec_a = ((64, ), (96, 128), (16, 32), (32,))
filter_spec_b = ((64, ), (96,), (16,), (32,))
filter_spec_c = ((128, ), (192, 256), (32, 64), (64,))

In [22]:
class VggInceptBlock2(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(VggInceptBlock2, self).__init__(**kwargs)
    def build(self, input_shape):
        self.inceptv2 = InceptionV2Block(filter_spec_a)
        self.inceptv2_1 = InceptionV2Block(filter_spec_a)
        self.inceptv2_2 = InceptionV2Block(filter_spec_a)
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
    def call(self, inputs):
        x = self.inceptv2(inputs)
        x = self.inceptv2_1(x)
        x = self.inceptv2_2(x)
        x = self.max_pool(x)
        return x

class VggInceptBlock3(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(VggInceptBlock3, self).__init__(**kwargs)
    def build(self, input_shape):
        self.inceptv3 = InceptionV3Block(filter_spec_c)
        self.inceptv3_1 = InceptionV3Block(filter_spec_c)
        self.inceptv3_2 = InceptionV3Block(filter_spec_c)
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2),  strides=(2, 2), padding='valid')
    def call(self, inputs):
        x = self.inceptv3(inputs)
        x = self.inceptv3_1(x)
        x = self.inceptv3_2(x)
        x = self.max_pool(x)
        return x

In [23]:
class InceptVgg16(tf.keras.Model):
    def __init__(self, include_top= True, pooling='max', classes=1000, **kwargs):
        super(InceptVgg16, self).__init__(**kwargs)
        self.block1 = VggBlockConvBN2(64)
        self.block2 = VggBlockConvBN2(128)
        self.block3 = VggInceptBlock2()
        self.block4 = VggBlockConvBN3(512)
        self.block5 = VggInceptBlock3()

        self.include_top = include_top
        self.pooling = pooling
        self.classes = classes

        if self.include_top:
            self.flatten = tf.keras.layers.Flatten()
            self.dense1 = tf.keras.layers.Dense(4096)
            self.bn1 = tf.keras.layers.BatchNormalization()
            self.dense2 = tf.keras.layers.Dense(4096)
            self.bn2 = tf.keras.layers.BatchNormalization()
            self.dense3 = tf.keras.layers.Dense(self.classes, activation='softmax')
        else:
            if self.pooling:
                self.pooling_layer = tf.keras.layers.GlobalMaxPool2D()
            else:
                self.pooling_layer = tf.keras.layers.GlobalAveragePooling2D()
    
    def call(self, inputs):
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)

        if self.include_top:
            x = self.flatten(x)
            x = self.dense1(x)
            x = self.bn1(x)
            x = self.dense2(x)
            x = self.bn2(x)
            x = self.dense3(x)
            return x
        else:
            return self.pooling_layer(x)



In [24]:
tf.keras.backend.clear_session()
my_incepVgg = InceptVgg16(include_top=False)
output = my_incepVgg(tf.random.normal((1, 224, 224, 1)))
my_incepVgg.summary()

Model: "incept_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg_block_conv_b_n2 (VggBloc multiple                  37824     
_________________________________________________________________
vgg_block_conv_b_n2_1 (VggBl multiple                  221952    
_________________________________________________________________
vgg_incept_block2 (VggIncept multiple                  482032    
_________________________________________________________________
vgg_block_conv_b_n3 (VggBloc multiple                  5902848   
_________________________________________________________________
vgg_incept_block3 (VggIncept multiple                  1735968   
_________________________________________________________________
global_max_pooling2d (Global multiple                  0         
Total params: 8,380,624
Trainable params: 8,368,240
Non-trainable params: 12,384
_______________________________________

# Lecture Template




In [25]:
import tensorflow.keras.backend as K

## 導入InceptionV2-有BatchNormalization的Convolution


In [26]:
def Conv2d_bn(x,filters,kernel_size,padding='same',strides=(1, 1),normalizer=True,activation='relu',name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = tf.keras.layers.Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(x)
    if normalizer:
        x = tf.keras.layers.BatchNormalization(axis=bn_axis, scale=False,  name=bn_name)(x)
    if activation:
        x = tf.keras.layers.Activation(activation, name=act_name)(x)
    return x

In [27]:
tf.keras.backend.clear_session()
img_input = tf.keras.layers.Input(shape=(224,224,1))
x = Conv2d_bn(img_input, 64, (3, 3))
model = tf.keras.Model(img_input, x)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      576       
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 64)      192       
_________________________________________________________________
activation (Activation)      (None, 224, 224, 64)      0         
Total params: 768
Trainable params: 640
Non-trainable params: 128
_________________________________________________________________


## 參考上圖搭建 



In [28]:
def InceptionV1_block(x, specs,channel_axis, name):
    (br0, br1, br2, br3) = specs   # ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0")

    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 3), name=name+"_Branch_1_1")

    '''Branch_2'''
    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2")
    branch_2 = Conv2d_bn(branch_2, br2[1], (3, 3), name=name+"_Branch_2_1")

    '''Branch_3'''
    branch_3 = tf.keras.layers.MaxPool2D(pool_size=(3, 3,), strides=(1, 1), padding='same')(x)
    branch_3 = Conv2d_bn(branch_3, br3[0], (1, 1), name=name+"_Branch_3")
    x = tf.keras.layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated")
    return x

## 測試

In [29]:
tf.keras.backend.clear_session()
img_input = tf.keras.layers.Input(shape=(224,224,1))
x=InceptionV1_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
model = tf.keras.Model(img_input, x)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
Block_1_Branch_1_conv (Conv2D)  (None, 224, 224, 96) 96          input_1[0][0]                    
__________________________________________________________________________________________________
Block_1_Branch_2_conv (Conv2D)  (None, 224, 224, 16) 16          input_1[0][0]                    
__________________________________________________________________________________________________
Block_1_Branch_1_bn (BatchNorma (None, 224, 224, 96) 288         Block_1_Branch_1_conv[0][0]      
_______________________________________________________________________________________

In [30]:
in_ch = 1
(1*1*in_ch)*64 + 64*3 + \
(1*1*in_ch)*96 + 96*3 + (3*3*96)*128 + 128*3 + \
(1*1*in_ch)*16 + 16*3 + (5*5*16)*32 + 32*3 + \
(1*1*in_ch)*32 + 32*3 , (64*2 + 96*2 + 128*2 + 16*2 + 32*2 + 32*2 )

(124704, 736)

## 將 InceptionV1_block中n*n卷積改為1 x n+n x 1

In [31]:
def InceptionV3_block(x, specs, channel_axis, name):
    (br0, br1, br2, br3) = specs   # ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0")

    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (1, 3), name=name+"_Branch_1_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 1), name=name+"_Branch_1_2")

    '''Branch_2'''
    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2")
    branch_2 = Conv2d_bn(branch_2, br2[1], (1, 3), name=name+"_Branch_2_1")
    branch_2 = Conv2d_bn(branch_2, br2[1], (3, 1), name=name+"_Branch_2_2")
    '''Branch_3'''
    branch_3 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_3 = Conv2d_bn(branch_3, br3[0], (1, 1), name=name+"_Branch_3")

    x = tf.keras.layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated")
    return x

## 測試

In [32]:
tf.keras.backend.clear_session()
img_input = tf.keras.layers.Input(shape=(224,224,1))
x=InceptionV3_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
print(x)

model = tf.keras.Model(img_input, x)
model.summary()

Tensor("Block_1_Concatenated/concat:0", shape=(None, 224, 224, 256), dtype=float32)
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
Block_1_Branch_1_conv (Conv2D)  (None, 224, 224, 96) 96          input_1[0][0]                    
__________________________________________________________________________________________________
Block_1_Branch_2_conv (Conv2D)  (None, 224, 224, 16) 16          input_1[0][0]                    
__________________________________________________________________________________________________
Block_1_Branch_1_bn (BatchNorma (None, 224, 224, 96) 288         Block_1_Branch_1_conv[0][0]      
___

## 額外練習

## 將VGG16 Block_3中的Convolution全部改為InceptionV1_block
## Block_5中的Convolution全部改為InceptionV3_block
## 並將所有Convolution改為Conv2d_bn

#### 原vgg16架構

In [33]:


def VGG16(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

   
    return model



#### 修改後

In [34]:
def VGG16_Inception(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    '''修改模型'''
    img_input = tf.keras.layers.Input(shape=input_shape)

    #Block1
    x = Conv2d_bn(img_input, 64, (3, 3), name='block1_conv1')
    x = Conv2d_bn(x, 64, (3, 3), name='block1_conv2')
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2d_bn(x, 128, (3, 3), name='block2_conv1')
    x = Conv2d_bn(x, 128, (3, 3), name='block2_conv2')
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = InceptionV1_block(x, filter_spec_a, 3, name='InV1_block_1')
    x = InceptionV1_block(x, filter_spec_a, 3, name='InV1_block_2')
    x = InceptionV1_block(x, filter_spec_a, 3, name='InV1_block_3')
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='InceptionV1_block3_pool')(x)

    # Block 4
    x = Conv2d_bn(x, 512, (3, 3), name='block4_conv1')
    x = Conv2d_bn(x, 512, (3, 3), name='block4_conv2')
    x = Conv2d_bn(x, 512, (3, 3), name='block4_conv3')
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = InceptionV3_block(x, filter_spec_c, 3, name='InV3_block_1')
    x = InceptionV3_block(x, filter_spec_c, 3, name='InV3_block_2')
    x = InceptionV3_block(x, filter_spec_c, 3, name='InV3_block_3')
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = tf.keras.layers.Flatten(name='flatten')(x)
        x = tf.keras.layers.Dense(4096, activation='relu', name='fc1')(x)
        x = tf.keras.Dense(4096, activation='relu', name='fc2')(x)
        x = tf.keras.layers.Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = tf.keras.layers.GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = tf.keras.layers.GlobalMaxPooling2D()(x)

    # Create model.
    model = tf.keras.Model(img_input, x, name='vgg16_inception')

    return model



In [35]:
model = VGG16_Inception(include_top=False)
model.summary()

Model: "vgg16_inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
block1_conv1_conv (Conv2D)      (None, 224, 224, 64) 576         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 224, 224, 64) 192         block1_conv1_conv[0][0]          
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 224, 224, 64) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________

In [36]:
input_map = 512
#branch0
b0 = input_map*1*128 

#branch1
b1 = (input_map*1*192, 192*3*256, 256*3*256)

#branch2
b2 = (input_map*1*32, 32*3*64, 64*3*64) 

#branch3
b3 = (input_map*1*64)
print(f'b0:{b0}, b1:{b1}, b2:{b2}, b3:{b3}')
filter_spec_c = ((128, ), (192, 256), (32, 64), (64,))

b0:65536, b1:(98304, 147456, 196608), b2:(16384, 6144, 12288), b3:32768


In [37]:
input_map = 256
#branch0
b0 = input_map*1*64 

#branch1
b1 = (input_map*1*96, 96*9*128, 128*9*128)

#branch2
b2 = (input_map*1*16, 16*9*32, 32*9*32) 

#branch3
b3 = (input_map*1*32)
print(f'b0:{b0}, b1:{b1}, b2:{b2}, b3:{b3}')
filter_spec_b = ((64, ), (96,), (16,), (32,))

b0:16384, b1:(24576, 110592, 147456), b2:(4096, 4608, 9216), b3:8192


In [38]:
input_map = 128
#branch0
b0 = input_map*1*64 

#branch1
b1 = (input_map*1*96, 96*9*128, 128*9*128)

#branch2
b2 = (input_map*1*16, 16*9*32, 32*9*32) 

#branch3
b3 = (input_map*1*32)
print(f'b0:{b0}, b1:{b1}, b2:{b2}, b3:{b3}')
filter_spec_a = ((64, ), (96, 128), (16, 32), (32,))

b0:8192, b1:(12288, 110592, 147456), b2:(2048, 4608, 9216), b3:4096
